In [3]:
import tensorflow as tf

## 0.简介
通过层层堆叠全连接层,保证前一层的输出节点数与当前层的输入节点数匹配，即可堆叠出任意层数的网络。我们把这种由神经元相互连接而成的网络叫做神经网络。如下图所示,通过堆叠4个全连接层,可以获得层数为4的神经网络,由于每层为全连接层,称为全连接网络。

![](https://github.com/zfhxi/Learn_tensorflow/blob/master/ch06-%C9%F1%BE%AD%CD%F8/img/05.png?raw=true)

设计全连接网络时，遵循约束：
隐藏层1的输入节点数需和数据的实际特征长度匹配，每层的输入层节点数与上一层输出节点数匹配，输出层的激活函数和节点数需要根据任务的具体设定进行设计。

至于与哪一组超参数是最优的，这需要很多的领域经验知识和大量的实验尝试，或者可以通过AutoML技术搜索出较优设定。

## 1.张量方式实现
实现上图的全连接网络：

### 1)定义参数张量

In [4]:
# 隐藏层1
w1,b1=tf.Variable(tf.random.truncated_normal([784,256],stddev=0.1)),tf.Variable(tf.zeros([256]))
# 隐藏层2
w2,b2=tf.Variable(tf.random.truncated_normal([256,128],stddev=0.1)),tf.Variable(tf.zeros([128]))
# 隐藏层3
w3,b3=tf.Variable(tf.random.truncated_normal([128,64],stddev=0.1)),tf.Variable(tf.zeros([64]))
# 输出层
w4,b4=tf.Variable(tf.random.truncated_normal([64,10],stddev=0.1)),tf.Variable(tf.zeros([10]))

### 2)前向

In [5]:
x=tf.random.truncated_normal([16,28*28],mean=0.5,stddev=0.1)
with tf.GradientTape() as tape: # 梯度记录器
    # x:[b,28*28]
    # 隐藏层1：[b,28*28] -> [b,256]
    h1=x@w1+b1
    h1=tf.nn.relu(h1)
    # 隐藏层2：[b,256] -> [b,128]
    h2=h1@w2+b2
    h2=tf.nn.relu(h2)
    # 隐藏层3：[b,128] -> [b,64]
    h3=h2@w3+b3
    h3=tf.nn.relu(h3)
    # 输出层：[b,64] -> [b,10]
    h4=h3@w4+b4
print(h4.shape)

(16, 10)


最后一层是否需要添加激活函数通常视具体的任务而定,这里加不加都可以。
在使用TensorFlow自动求导功能计算梯度时,需要将前向计算过程放置在`tf.GradientTape()`环境中,从而利用`GradientTape`对象的`gradient()`方法自动求解参数的梯度,并利用`optimizers`对象更新参数。

## 3.层方式实现

In [6]:
# 导入常用网络层
from tensorflow.keras import layers,Sequential
fc1=layers.Dense(256,activation=tf.nn.relu) # 隐藏层1
fc2=layers.Dense(128,activation=tf.nn.relu) # 隐藏层2
fc3=layers.Dense(64,activation=tf.nn.relu) # 隐藏层3
fc4=layers.Dense(10,activation=None) # 输出层

# 前向
x=tf.random.normal([4,28*28])
h1=fc1(x)
h2=fc2(h1)
h3=fc3(h2)
h4=fc4(h3)
print(h4.shape)

(4, 10)


或者将多个全连接层封装为一个网络类

In [7]:
model=Sequential([
    layers.Dense(256,activation=tf.nn.relu), # 隐藏层1
    layers.Dense(128,activation=tf.nn.relu), # 隐藏层2
    layers.Dense(64,activation=tf.nn.relu), # 隐藏层3
    layers.Dense(10,activation=None) # 输出层
])
out=model(x)
print(out.shape)

(4, 10)


## 4.优化目标
上面完成了前向计算的一部分，最后一步便是计算误差：

${\cal L}=g(f_\theta(x),y)$

* $f_\theta(\cdot)$表示利用\theta参数化的神经网络模型
* $g(\cdot)$称之为误差函数，描述网络预测值$f_\theta(x)$与真实标签$y$之间的距离

希望通过在训练集$\mathbb D^\mathrm{train}$上学习到一组参数$\theta$使得训练的误差$\mathcal L$最小：

$\theta^*=\underbrace{\rm arg\ min}_\theta\ g(f_\theta\lgroup x\rgroup,y), x\in\mathbb D^{\rm train}$

针对该最小优化问题，一般采用误差反向传播算法求解，并利用梯度下降算法迭代更新参数：

$\theta^\prime=\theta-\eta\cdot\nabla_\theta{\cal L}$

In [ ]:
import os
pid=os.getpid()
!kill -9 $pid


